In [6]:
from functions.sudoku import Sudoku

In [7]:
values = [[1, 7, 3, 5, 4, 2, 8, 9, 6],
 [2, 6, 9, 8, 1, 7, 4, 5, 3],
 [8, 4, 5, 9, 3, 6, 1, 2, 7],
 [5, 3, 6, 7, 9, 1, 2, 4, 8],
 [7, 8, 1, 4, 2, 5, 6, 3, 9],
 [9, 2, 4, 6, 8, 3, 7, 1, 5],
 [4, 9, 7, 1, 5, 8, 3, 6, 2],
 [6, 5, 2, 3, 7, 4, 9, 8, 1],
 [3, 1, 8, 2, 6, 9, 5, 7, 4]]

In [8]:
sudoku = Sudoku(values)

In [9]:
class GenSolver:
    def __init__(self, sudoku, pop_size = 10):
        self.initial_sudoku = sudoku
        self.pop_size = pop_size
        self.population = [Sudoku(self.fill_sudoku_random(sudoku)) for _ in range(self.pop_size)]
        
    def fill_row_random(self, row):
        import random
        new_row = list()
        missing_values = set(range(1,10)).difference(set(row))
        rearranged = random.sample(missing_values, len(missing_values))
        for i in range(9):
            if row[i] == 0:
                new_row.append(rearranged.pop())
            else:
                new_row.append(row[i])
        return new_row
        
    def fill_sudoku_random(self, sudoku):
        import random       
        filled_sudoku = list()
        
        for row in sudoku.values:
            filled_sudoku.append(self.fill_row_random(row))
        return filled_sudoku
    
    def crossover(self, pa, ma):
        import random
        swapped_rows = random.sample(range(9), random.randint(1,8))
        bro = list()
        sis = list()
        for i in range(9):
            if i in swapped_rows:
                bro.append(ma.values[i])
                sis.append(pa.values[i])
            else:
                bro.append(pa.values[i])
                sis.append(ma.values[i])
        return Sudoku(bro), Sudoku(sis)
    
    def mutate(self, sudoku):
        import random
        mutated_rows = random.sample(range(9), random.randint(1,9))
        mutated = list()
        for i in range(9):
            if i in mutated_rows:
                mutated.append(self.fill_row_random(self.initial_sudoku.values[i]))
            else:
                mutated.append(sudoku.values[i])
        return Sudoku(mutated)
    
    def tournament_select(self):
        import random
        competitors = random.sample(self.population, 2)
        if competitors[0].cost < competitors[1].cost:
            return competitors[0]
        else:
            return competitors[1]
    
    def make_new_population(self, mutation_prob = 0.4):
        import random
        import numpy as np
        new_pop = []
        # Keep the best two - this guarantees that the new population is no worse as the previous one
        new_pop.extend([self.population[x] for x in np.argsort([sud.cost for sud in self.population])[:2]])
        for i in range(int(self.pop_size / 2) - 1):
            ma = self.tournament_select()
            pa = self.tournament_select()
            #If ma and pa are identical, breed with a random one instead sometimes
            #if ma.values == pa.values and random.random() < 0.5:
            #    pa = Sudoku(self.fill_sudoku_random(self.initial_sudoku))
            bro, sis = self.crossover(ma, pa)
            if random.random() < mutation_prob: 
                bro = self.mutate(bro)
            if random.random() < mutation_prob: 
                sis = self.mutate(sis)
            new_pop.extend([bro,sis])
        self.population = new_pop

In [10]:
easy = [[0,0,0,0,0,8,5,4,3],
       [0,2,5,6,1,0,7,0,9],
       [0,0,0,0,0,0,0,2,6],
       [0,3,0,0,0,2,0,6,7],
       [0,0,0,5,7,1,0,0,0],
       [7,8,0,4,0,0,0,1,0],
       [2,7,0,0,0,0,0,0,0],
       [1,0,9,0,4,3,2,5,0],
       [4,5,3,8,0,0,0,0,0]]

In [11]:
gs = GenSolver(Sudoku(easy), pop_size=100)

In [12]:
min([x.cost for x in gs.population])

34

In [13]:
while (min([x.cost for x in gs.population]) > 0):
    gs.make_new_population()
    print(min([x.cost for x in gs.population]))

28
28
28
27
27
24
24
24
24
24
24
24
24
24
22
22
22
22
20
20
20
20
20
20
20
20
20
20
20
19
19
19


KeyboardInterrupt: 

In [233]:
len(gs.population)

98

In [162]:
gs.population[1].values

[[6, 8, 3, 4, 2, 1, 5, 7, 9],
 [5, 2, 3, 9, 4, 8, 7, 6, 1],
 [7, 9, 4, 6, 3, 8, 5, 1, 2],
 [4, 5, 6, 1, 7, 2, 9, 8, 3],
 [2, 5, 3, 9, 7, 6, 4, 1, 8],
 [2, 1, 8, 6, 7, 4, 3, 9, 5],
 [3, 2, 9, 8, 1, 7, 5, 4, 6],
 [4, 6, 9, 5, 7, 3, 2, 1, 8],
 [8, 7, 6, 5, 4, 2, 1, 3, 9]]

In [163]:
gs.crossover(gs.population[0], gs.population[1])

(<__main__.Sudoku at 0x7fa4f82c4160>, <__main__.Sudoku at 0x7fa4f82c4f28>)

In [165]:
gs.mutate(gs.population[0]).values

[[6, 8, 3, 9, 2, 1, 5, 7, 4],
 [5, 3, 4, 9, 1, 8, 2, 6, 7],
 [7, 3, 4, 6, 5, 9, 1, 8, 2],
 [4, 2, 6, 1, 3, 5, 7, 8, 9],
 [8, 5, 6, 7, 3, 9, 2, 1, 4],
 [8, 1, 9, 6, 2, 4, 3, 7, 5],
 [3, 1, 9, 4, 8, 7, 5, 2, 6],
 [7, 6, 4, 5, 1, 3, 2, 9, 8],
 [2, 5, 6, 8, 4, 7, 1, 3, 9]]